In [1]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURAÇÃO ---
caminho_arquivo = r"E:\Estudos_Cybersecurity\Datasets_Cybersecurity\NSL-KDD\nsl-kdd\KDDTrain+.txt"

colunas = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
    'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
    'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
    'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate',
    'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
    'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate',
    'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'class', 'difficulty_level'
]

# Carrega o dataset
print("Carregando dados para limpeza...")
df = pd.read_csv(caminho_arquivo, names=colunas, index_col=False)

# Remove a coluna 'difficulty_level' (não ajuda a IA a detectar ataques, é apenas um metadado)
df = df.drop('difficulty_level', axis=1)

print(f"Dataset pronto: {df.shape}")
print("Colunas de texto (categóricas) que precisamos transformar:")
print(df.select_dtypes(include=['object']).columns.tolist())

Carregando dados para limpeza...
Dataset pronto: (125973, 42)
Colunas de texto (categóricas) que precisamos transformar:
['protocol_type', 'service', 'flag', 'class']


In [2]:
# --- 1. CRIANDO O ALVO BINÁRIO ---
# Se for 'normal' vira 0, qualquer outra coisa (ataque) vira 1
df['target'] = df['class'].apply(lambda x: 0 if x == 'normal' else 1)

print("--- Transformação do Alvo Concluída ---")
print("Exemplos da transformação:")
print(df[['class', 'target']].head(10)) # Mostra lado a lado para conferir

# --- 2. VERIFICANDO O EQUILÍBRIO ---
contagem = df['target'].value_counts()
print("\nNovas quantidades:")
print(f"0 (Normal): {contagem[0]}")
print(f"1 (Ataque): {contagem[1]}")

--- Transformação do Alvo Concluída ---
Exemplos da transformação:
     class  target
0   normal       0
1   normal       0
2  neptune       1
3   normal       0
4   normal       0
5  neptune       1
6  neptune       1
7  neptune       1
8  neptune       1
9  neptune       1

Novas quantidades:
0 (Normal): 67343
1 (Ataque): 58630


In [ ]:
# --- 1. REMOVENDO A ANTIGA COLUNA 'CLASS' ---
# Já temos a 'target', então não precisamos mais do texto original
df = df.drop('class', axis=1)

# --- 2. ONE-HOT ENCODING (A Mágica) ---
# Selecionamos as colunas que ainda são texto
cols_categoricas = ['protocol_type', 'service', 'flag']

print(f"Colunas antes: {df.shape[1]}")

# O pandas transforma tudo automaticamente
df = pd.get_dummies(df, columns=cols_categoricas)

print(f"Colunas depois: {df.shape[1]}")

# Mostra como ficou a 'cabeça' do dataset (colunas novas)
# Para ver apenas as primeiras colunas de protocolo para confirmar
colunas_novas = [col for col in df.columns if 'protocol' in col]
print("\nExemplo das novas colunas geradas:")
display(df[colunas_novas].head())

Colunas antes: 42
Colunas depois: 123

Exemplo das novas colunas geradas:


,protocol_type_icmp,protocol_type_tcp,protocol_type_udp
0,False,True,False
1,False,False,True
2,False,True,False
3,False,True,False
4,False,True,False


In [ ]:
# %pip install scikit-learn  # (Descomente se der erro de módulo)
from sklearn.preprocessing import MinMaxScaler

# --- 1. DEFININDO QUEM VAI SER ESCALADO ---
# Para escalar TUDO, menos a nossa resposta ('target')
colunas_para_escalar = df.columns.drop('target')

print("Iniciando normalização (0 a 1)...")

# --- 2. APLICANDO A RÉGUA (MinMaxScaler) ---
scaler = MinMaxScaler()
df[colunas_para_escalar] = scaler.fit_transform(df[colunas_para_escalar])

print("✅ Normalização concluída!")

# --- 3. PROVA REAL ---
# Para ver os valores máximos. Todos devem ser 1.0 agora.
print("\nVerificando máximos das primeiras 5 colunas numéricas:")
print(df[colunas_para_escalar].max().head())

print("\nVerificando máximos das colunas de bytes (que eram gigantes):")
print(df[['src_bytes', 'dst_bytes']].max())

Iniciando normalização (0 a 1)...
✅ Normalização concluída!

Verificando máximos das primeiras 5 colunas numéricas:
duration          1.0
src_bytes         1.0
dst_bytes         1.0
land              1.0
wrong_fragment    1.0
dtype: float64

Verificando máximos das colunas de bytes (que eram gigantes):
src_bytes    1.0
dst_bytes    1.0
dtype: float64


In [5]:
# --- 4. SALVANDO O ARQUIVO PROCESSADO ---
caminho_salvar = r"E:\Estudos_Cybersecurity\Datasets_Cybersecurity\train_ready.csv"

df.to_csv(caminho_salvar, index=False)
print(f"💾 Arquivo salvo com sucesso em: {caminho_salvar}")

💾 Arquivo salvo com sucesso em: E:\Estudos_Cybersecurity\Datasets_Cybersecurity\train_ready.csv
